In [42]:
import pandas as pd
import numpy as np
import re
from scipy import stats
import matplotlib.pyplot as plt
import wordcloud 
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import warnings 
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [43]:
df1= pd.read_csv('1- The Lancet Digital Health.csv')
df2= pd.read_csv('2- GigaScience.csv')
df3= pd.read_csv('3- npj Digital Medicine.csv')
df4= pd.read_csv('4- Medical Image Analysis.csv')
df5= pd.read_csv('5- Implementation Science.csv')
df6= pd.read_csv('6- MMWR Recommendations and Reports.csv')
df7= pd.read_csv('7 - Morbidity and Mortality Weekly Report.csv')
df8= pd.read_csv('8- MMWR Surveillance Summaries.csv')
df9= pd.read_csv('9- Journal of the American Medical Informatics Association JAMIA.csv')
df10= pd.read_csv('10- Journal of Medical Internet Research.csv')

In [44]:
articles_df = pd.concat(([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]), ignore_index=True)

In [48]:
pd.options.display.max_colwidth =100
pd.set_option("display.max_columns", None)
articles_df = articles_df.drop(['authorfullnames','author(s)id','doi','link','eid','source','publicationstage','publisher'],axis=1)
articles_df

,authors,title,year,sourcetitle,citedby,abstract,indexkeywords,documenttype
0,Nagaraj D.; Khandelwal P.; Steyaert S.; Gevaert O.,Augmenting digital twins with federated learning in medicine,2023,The Lancet Digital Health,0,[No abstract available],Medicine; medicine,Note
1,NaN,Correction to Lancet Digit Health 2022; 4: e862–72 (The Lancet Digital Health (2022) 4(12) (e862...,2023,The Lancet Digital Health,0,"Fainberg HP, Oldham JM, Molyneaux PL, et al. Forced vital capacity trajectories in patients with...",erratum,Erratum
2,NaN,Erratum: Correction to Lancet Digit Health 2022; 4: e497-506 (The Lancet. Digital health (2022) ...,2022,The Lancet. Digital health,0,[No abstract available],erratum,Erratum
3,Vayena E.; Blasimme A.; Sugarman J.,Decentralised clinical trials: ethical opportunities and challenges,2023,The Lancet Digital Health,0,"Fuelled by adaptations to clinical trial implementation during the COVID-19 pandemic, decentrali...","COVID-19; Ethics, Research; Humans; Informed Consent; Pandemics; Research Personnel; COVID-19; D...",Review
4,McLean K.A.; Knight S.R.; Diehl T.M.; Varghese C.; Ng N.; Potter M.A.; Zafar S.N.; Bouamrane M.-...,Readiness for implementation of novel digital health interventions for postoperative monitoring:...,2023,The Lancet Digital Health,1,An increasing number of digital health interventions (DHIs) for remote postoperative monitoring ...,Telemedicine; Health care; Co-authorships; Comprehensive evaluation; Health interventions; High ...,Review
...,...,...,...,...,...,...,...,...
15449,Emmert M.; Halling F.; Meier F.,Evaluations of dentists on a German physician rating website: An analysis of the ratings,2015,Journal of Medical Internet Research,33,Background: Physician rating websites have been gaining in importance in both practice and resea...,Adult; Age Distribution; Dentists; Female; Germany; Humans; Internet; Male; Middle Aged; Physici...,Article
15450,Essa-Hadad J.; Linn S.; Rafaeli S.,A web-based program to increase knowledge and reduce cigarette and nargila smoking among Arab un...,2015,Journal of Medical Internet Research,17,"Background: Among Arab citizens in Israel, cigarette and nargila (hookah, waterpipe) smoking is ...",Adolescent; Adult; Arabs; Feedback; Female; Focus Groups; Humans; Intention; Internet; Israel; M...,Article
15451,Silver M.P.,Patient perspectives on online health information and communication with doctors: A qualitative ...,2015,Journal of Medical Internet Research,140,Background: As health care systems around the world shift toward models that emphasize self-care...,Aged; Attitude to Computers; Attitude to Health; Communication; Consumer Health Information; Fem...,Article
15452,Nasi G.; Cucciniello M.; Guerrazzi C.,The role of mobile technologies in health care processes: The case of cancer supportive care,2015,Journal of Medical Internet Research,99,Background: Health care systems are gradually moving toward new models of care based on integrat...,Cell Phones; Humans; Information Systems; Mobile Applications; Neoplasms; Palliative Care; Palli...,Article


In [49]:
articles_df.columns=articles_df.columns.str.replace(" ","").str.lower()

In [50]:
articles_df.documenttype.value_counts()

Article              10514
Review                2033
Note                  1205
Erratum                723
Book chapter           323
Conference paper       186
Editorial              185
Letter                 131
Short survey           104
Book                    23
Data paper              18
Conference review        6
Retracted                3
Name: documenttype, dtype: int64

In [51]:
articles_df.documenttype=np.where(articles_df["documenttype"].str.contains("Article|Review"), "Article", "Not Article")

In [52]:
articles_df.documenttype.value_counts()

Article        12547
Not Article     2907
Name: documenttype, dtype: int64

In [53]:
articles_df1 = articles_df.loc[articles_df.documenttype=='Article']
articles_df2 = articles_df.loc[articles_df.documenttype=='Review']
articles_df = pd.concat(([articles_df1,articles_df2]), ignore_index=True)

In [54]:
articles_df

,authors,title,year,sourcetitle,citedby,abstract,indexkeywords,documenttype
0,Vayena E.; Blasimme A.; Sugarman J.,Decentralised clinical trials: ethical opportunities and challenges,2023,The Lancet Digital Health,0,"Fuelled by adaptations to clinical trial implementation during the COVID-19 pandemic, decentrali...","COVID-19; Ethics, Research; Humans; Informed Consent; Pandemics; Research Personnel; COVID-19; D...",Article
1,McLean K.A.; Knight S.R.; Diehl T.M.; Varghese C.; Ng N.; Potter M.A.; Zafar S.N.; Bouamrane M.-...,Readiness for implementation of novel digital health interventions for postoperative monitoring:...,2023,The Lancet Digital Health,1,An increasing number of digital health interventions (DHIs) for remote postoperative monitoring ...,Telemedicine; Health care; Co-authorships; Comprehensive evaluation; Health interventions; High ...,Article
2,Jiang R.; Noble S.; Sui J.; Yoo K.; Rosenblatt M.; Horien C.; Qi S.; Liang Q.; Sun H.; Calhoun V...,Associations of physical frailty with health outcomes and brain structure in 483 033 middle-aged...,2023,The Lancet Digital Health,1,Background: Physical frailty is a state of increased vulnerability to stressors and is associate...,"Aged; Biological Specimen Banks; Brain; Frailty; Humans; Middle Aged; Outcome Assessment, Health...",Article
3,Oikonomou E.K.; Spatz E.S.; Suchard M.A.; Khera R.,Individualising intensive systolic blood pressure reduction in hypertension using computational ...,2022,The Lancet Digital Health,2,Background: The cardiovascular benefits of intensive systolic blood pressure control vary across...,"Aged; Antihypertensive Agents; Blood Pressure; Cardiovascular Diseases; Diabetes Mellitus, Type ...",Article
4,Cheung C.Y.; Ran A.R.; Wang S.; Chan V.T.T.; Sham K.; Hilal S.; Venketasubramanian N.; Cheng C.-...,A deep learning model for detection of Alzheimer's disease based on retinal photographs: a retro...,2022,The Lancet Digital Health,11,"Background: There is no simple model to screen for Alzheimer's disease, partly because the diagn...",Alzheimer Disease; Amyloid beta-Peptides; Case-Control Studies; Deep Learning; Humans; Retrospec...,Article
...,...,...,...,...,...,...,...,...
12542,Emmert M.; Halling F.; Meier F.,Evaluations of dentists on a German physician rating website: An analysis of the ratings,2015,Journal of Medical Internet Research,33,Background: Physician rating websites have been gaining in importance in both practice and resea...,Adult; Age Distribution; Dentists; Female; Germany; Humans; Internet; Male; Middle Aged; Physici...,Article
12543,Essa-Hadad J.; Linn S.; Rafaeli S.,A web-based program to increase knowledge and reduce cigarette and nargila smoking among Arab un...,2015,Journal of Medical Internet Research,17,"Background: Among Arab citizens in Israel, cigarette and nargila (hookah, waterpipe) smoking is ...",Adolescent; Adult; Arabs; Feedback; Female; Focus Groups; Humans; Intention; Internet; Israel; M...,Article
12544,Silver M.P.,Patient perspectives on online health information and communication with doctors: A qualitative ...,2015,Journal of Medical Internet Research,140,Background: As health care systems around the world shift toward models that emphasize self-care...,Aged; Attitude to Computers; Attitude to Health; Communication; Consumer Health Information; Fem...,Article
12545,Nasi G.; Cucciniello M.; Guerrazzi C.,The role of mobile technologies in health care processes: The case of cancer supportive care,2015,Journal of Medical Internet Research,99,Background: Health care systems are gradually moving toward new models of care based on integrat...,Cell Phones; Humans; Information Systems; Mobile Applications; Neoplasms; Palliative Care; Palli...,Article


In [55]:
articles_df.isnull().sum().to_dict()

{'authors': 242,
 'title': 0,
 'year': 0,
 'sourcetitle': 0,
 'citedby': 0,
 'abstract': 0,
 'indexkeywords': 270,
 'documenttype': 0}

In [56]:
articles_df.shape

(12547, 8)

In [57]:
articles_df[['year']].value_counts()

year
2021    2368
2020    2188
2022    1801
2019    1413
2018    1113
2017     998
2023     943
2015     886
2016     837
dtype: int64

In [58]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12547 entries, 0 to 12546
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   authors        12305 non-null  object
 1   title          12547 non-null  object
 2   year           12547 non-null  int64 
 3   sourcetitle    12547 non-null  object
 4   citedby        12547 non-null  int64 
 5   abstract       12547 non-null  object
 6   indexkeywords  12277 non-null  object
 7   documenttype   12547 non-null  object
dtypes: int64(2), object(6)
memory usage: 784.3+ KB


In [59]:
articles_df.sourcetitle.value_counts()

Journal of Medical Internet Research                                                                      5808
Medical Image Analysis                                                                                    1553
Morbidity and Mortality Weekly Report                                                                     1486
Implementation Science                                                                                    1001
GigaScience                                                                                                895
npj Digital Medicine                                                                                       701
MMWR. Morbidity and mortality weekly report                                                                332
The Lancet Digital Health                                                                                  251
MMWR Recommendations and Reports                                                                           237
M

In [60]:
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

In [61]:
for col in articles_df.columns[:]:
     print(col,':',len(articles_df[col].unique()),'labels')

authors : 12213 labels
title : 12531 labels
year : 9 labels
sourcetitle : 17 labels
citedby : 414 labels
abstract : 11420 labels
indexkeywords : 12246 labels
documenttype : 1 labels


# Tokenization

In [62]:
def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)

In [63]:
articles_df=articles_df.drop('documenttype',axis=1)

In [64]:
articles_df

,authors,title,year,sourcetitle,citedby,abstract,indexkeywords
0,Vayena E.; Blasimme A.; Sugarman J.,Decentralised clinical trials: ethical opportunities and challenges,2023,The Lancet Digital Health,0,"Fuelled by adaptations to clinical trial implementation during the COVID-19 pandemic, decentrali...","COVID-19; Ethics, Research; Humans; Informed Consent; Pandemics; Research Personnel; COVID-19; D..."
1,McLean K.A.; Knight S.R.; Diehl T.M.; Varghese C.; Ng N.; Potter M.A.; Zafar S.N.; Bouamrane M.-...,Readiness for implementation of novel digital health interventions for postoperative monitoring:...,2023,The Lancet Digital Health,1,An increasing number of digital health interventions (DHIs) for remote postoperative monitoring ...,Telemedicine; Health care; Co-authorships; Comprehensive evaluation; Health interventions; High ...
2,Jiang R.; Noble S.; Sui J.; Yoo K.; Rosenblatt M.; Horien C.; Qi S.; Liang Q.; Sun H.; Calhoun V...,Associations of physical frailty with health outcomes and brain structure in 483 033 middle-aged...,2023,The Lancet Digital Health,1,Background: Physical frailty is a state of increased vulnerability to stressors and is associate...,"Aged; Biological Specimen Banks; Brain; Frailty; Humans; Middle Aged; Outcome Assessment, Health..."
3,Oikonomou E.K.; Spatz E.S.; Suchard M.A.; Khera R.,Individualising intensive systolic blood pressure reduction in hypertension using computational ...,2022,The Lancet Digital Health,2,Background: The cardiovascular benefits of intensive systolic blood pressure control vary across...,"Aged; Antihypertensive Agents; Blood Pressure; Cardiovascular Diseases; Diabetes Mellitus, Type ..."
4,Cheung C.Y.; Ran A.R.; Wang S.; Chan V.T.T.; Sham K.; Hilal S.; Venketasubramanian N.; Cheng C.-...,A deep learning model for detection of Alzheimer's disease based on retinal photographs: a retro...,2022,The Lancet Digital Health,11,"Background: There is no simple model to screen for Alzheimer's disease, partly because the diagn...",Alzheimer Disease; Amyloid beta-Peptides; Case-Control Studies; Deep Learning; Humans; Retrospec...
...,...,...,...,...,...,...,...
12542,Emmert M.; Halling F.; Meier F.,Evaluations of dentists on a German physician rating website: An analysis of the ratings,2015,Journal of Medical Internet Research,33,Background: Physician rating websites have been gaining in importance in both practice and resea...,Adult; Age Distribution; Dentists; Female; Germany; Humans; Internet; Male; Middle Aged; Physici...
12543,Essa-Hadad J.; Linn S.; Rafaeli S.,A web-based program to increase knowledge and reduce cigarette and nargila smoking among Arab un...,2015,Journal of Medical Internet Research,17,"Background: Among Arab citizens in Israel, cigarette and nargila (hookah, waterpipe) smoking is ...",Adolescent; Adult; Arabs; Feedback; Female; Focus Groups; Humans; Intention; Internet; Israel; M...
12544,Silver M.P.,Patient perspectives on online health information and communication with doctors: A qualitative ...,2015,Journal of Medical Internet Research,140,Background: As health care systems around the world shift toward models that emphasize self-care...,Aged; Attitude to Computers; Attitude to Health; Communication; Consumer Health Information; Fem...
12545,Nasi G.; Cucciniello M.; Guerrazzi C.,The role of mobile technologies in health care processes: The case of cancer supportive care,2015,Journal of Medical Internet Research,99,Background: Health care systems are gradually moving toward new models of care based on integrat...,Cell Phones; Humans; Information Systems; Mobile Applications; Neoplasms; Palliative Care; Palli...


In [65]:
articles_df['authors'] = articles_df['authors'].astype(str)
articles_df['indexkeywords'] = articles_df['indexkeywords'].astype(str)

In [66]:
articles_df['authors']=articles_df['authors'].apply(custom_tokenize)
articles_df['title']=articles_df['title'].apply(custom_tokenize)
articles_df['sourcetitle']=articles_df['sourcetitle'].apply(custom_tokenize)
articles_df['abstract']=articles_df['abstract'].apply(custom_tokenize)
articles_df['indexkeywords']=articles_df['indexkeywords'].apply(custom_tokenize)

# Lemmatization

In [67]:
def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma

In [68]:
lemmatizer = WordNetLemmatizer()
articles_df['authors']=articles_df['authors'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
articles_df['title']=articles_df['title'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
articles_df['sourcetitle']=articles_df['sourcetitle'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
articles_df['abstract']=articles_df['abstract'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
articles_df['indexkeywords']=articles_df['indexkeywords'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Getting rid of StopWords

In [69]:
stop_words=set(stopwords.words("english"))

In [70]:
articles_df['authors']=articles_df['authors'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))
articles_df['title']=articles_df['title'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))
articles_df['sourcetitle']=articles_df['sourcetitle'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))
articles_df['abstract']=articles_df['abstract'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))
articles_df['indexkeywords']=articles_df['indexkeywords'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))

# Removing Punctuations

In [71]:
import string
def remove_punct(text):
    """
        Remove the punctuation
    """
#     return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', "", text)
    return text.translate(str.maketrans('', '', string.punctuation))

In [72]:
articles_df['authors']=articles_df['authors'].apply(lambda x: remove_punct(x))
articles_df['title']=articles_df['title'].apply(lambda x: remove_punct(x))
articles_df['sourcetitle']=articles_df['sourcetitle'].apply(lambda x: remove_punct(x))
articles_df['abstract']=articles_df['abstract'].apply(lambda x: remove_punct(x))
articles_df['indexkeywords']=articles_df['indexkeywords'].apply(lambda x: remove_punct(x))

# Removing Special Characters

In [73]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [74]:
articles_df['authors']=articles_df['authors'].apply(lambda x: remove_special_characters(x))
articles_df['title']=articles_df['title'].apply(lambda x: remove_special_characters(x))
articles_df['sourcetitle']=articles_df['sourcetitle'].apply(lambda x: remove_special_characters(x))
articles_df['abstract']=articles_df['abstract'].apply(lambda x: remove_special_characters(x))
articles_df['indexkeywords']=articles_df['indexkeywords'].apply(lambda x: remove_special_characters(x))

# Cleaning all other noise

In [75]:
def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''},  # remove spaces at the beginning
        {r'^a-zA-z.,!?/:;\"\'\s': u''}
    ]
    for rule in rules:
        for (k, v) in rule.items():
            regex = re.compile(k) #compiles a regular expression pattern provided as a string into a regex pattern object
            text = regex.sub(v, text) # returns a string after replacing the matched pattern in a string with a replacement.
            text = text.rstrip() #removes any trailing characters (characters at the end a string)
    return text.lower() # get the unique format

In [76]:
articles_df['authors']=articles_df['authors'].apply(text_cleaner)
articles_df['title']=articles_df['title'].apply(text_cleaner)
articles_df['sourcetitle']=articles_df['sourcetitle'].apply(text_cleaner)
articles_df['abstract']=articles_df['abstract'].apply(text_cleaner)
articles_df['indexkeywords']=articles_df['indexkeywords'].apply(text_cleaner)

In [77]:
articles_df

,authors,title,year,sourcetitle,citedby,abstract,indexkeywords
0,vayena e blasimme a sugarman j,decentralised clinical trial ethical opportunity challenge,2023,the lancet digital health,0,fuelled adaptation clinical trial implementation covid19 pandemic decentralised clinical trial b...,covid19 ethics research humans informed consent pandemics research personnel covid19 digital dev...
1,mclean ka knight sr diehl tm varghese c ng n potter ma zafar sn bouamrane mm harrison em,readiness implementation novel digital health intervention postoperative monitoring systematic r...,2023,the lancet digital health,1,an increasing number digital health intervention dhis remote postoperative monitoring developed ...,telemedicine health care coauthorships comprehensive evaluation health intervention high quality...
2,jiang r noble s sui j yoo k rosenblatt m horien c qi s liang q sun h calhoun vd scheinost d,associations physical frailty health outcome brain structure 483 033 middleaged older adult popu...,2023,the lancet digital health,1,background physical frailty state increased vulnerability stressor associated serious health iss...,aged biological specimen banks brain frailty humans middle aged outcome assessment health care u...
3,oikonomou ek spatz es suchard ma khera r,individualising intensive systolic blood pressure reduction hypertension using computational tri...,2022,the lancet digital health,2,background the cardiovascular benefit intensive systolic blood pressure control vary across clin...,aged antihypertensive agents blood pressure cardiovascular diseases diabetes mellitus type 2 fem...
4,cheung cy ran ar wang s chan vtt sham k hilal s venketasubramanian n cheng cy sabanayagam c tham...,a deep learning model detection alzheimer s disease based retinal photograph retrospective multi...,2022,the lancet digital health,11,background there simple model screen alzheimer s disease partly diagnosis alzheimer s disease co...,alzheimer disease amyloid betapeptides casecontrol studies deep learning humans retrospective st...
...,...,...,...,...,...,...,...
12542,emmert m halling f meier f,evaluations dentist german physician rating website an analysis rating,2015,journal medical internet research,33,background physician rating website gaining importance practice research however evidence availa...,adult age distribution dentists female germany humans internet male middle aged physicians socia...
12543,essahadad j linn s rafaeli s,a webbased program increase knowledge reduce cigarette nargila smoking among arab university stu...,2015,journal medical internet research,17,background among arab citizen israel cigarette nargila hookah waterpipe smoking serious public h...,adolescent adult arabs feedback female focus groups humans intention internet israel male pilot ...
12544,silver mp,patient perspective online health information communication doctor a qualitative study patient 5...,2015,journal medical internet research,140,background as health care system around world shift toward model emphasize selfcare management i...,aged attitude computers attitude health communication consumer health information female humans ...
12545,nasi g cucciniello m guerrazzi c,the role mobile technology health care process the case cancer supportive care,2015,journal medical internet research,99,background health care system gradually moving toward new model care based integrated care proce...,cell phones humans information systems mobile applications neoplasms palliative care palliative ...


In [78]:
articles_df['indexkeywords'][0]

'covid19 ethics research humans informed consent pandemics research personnel covid19 digital device medical application patient treatment clinical trial decentralised digital tool outcome measure patientreported outcome physical contact research integrity research team safety integrity safety monitoring alertness clinical trial topic coronavirus disease 2019 decentralization decentralized clinical trial topic digital technology food drug administration human informed consent interpersonal communication miscellaneous named group patient right patientreported outcome practice guideline privacy research ethic research investigator research sponsor research subject research team review safety standardization wellbeing coronavirus disease 2019 pandemic personnel research ethic philosophical aspect'

In [83]:
def Vectorizing(x):
    vector = TfidfVectorizer()
    tokenized=sent_tokenize(x)
    tf_idf=vector.fit_transform(tokenized)
    return tf_idf.toarray()

In [84]:
articles_df['authors']=articles_df['authors'].apply(Vectorizing)
articles_df['title']=articles_df['title'].apply(Vectorizing)
articles_df['sourcetitle']=articles_df['sourcetitle'].apply(Vectorizing)
articles_df['abstract']=articles_df['abstract'].apply(Vectorizing)
articles_df['indexkeywords']=articles_df['indexkeywords'].apply(Vectorizing)

In [91]:
articles_df = pd.DataFrame(articles_df)
articles_df.head(3)

,authors,title,year,sourcetitle,citedby,abstract,indexkeywords
0,"[[0.5773502691896258, 0.5773502691896258, 0.5773502691896258]]","[[0.4082482904638631, 0.4082482904638631, 0.4082482904638631, 0.4082482904638631, 0.408248290463...",2023,"[[0.5, 0.5, 0.5, 0.5]]",0,"[[0.060302268915552716, 0.060302268915552716, 0.060302268915552716, 0.12060453783110543, 0.06030...","[[0.13018891098082389, 0.06509445549041194, 0.06509445549041194, 0.06509445549041194, 0.06509445..."
1,"[[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]]","[[0.2672612419124244, 0.2672612419124244, 0.2672612419124244, 0.2672612419124244, 0.267261241912...",2023,"[[0.5, 0.5, 0.5, 0.5]]",1,"[[0.06917144638660747, 0.06917144638660747, 0.06917144638660747, 0.06917144638660747, 0.06917144...","[[0.13245323570650439, 0.13245323570650439, 0.13245323570650439, 0.13245323570650439, 0.13245323..."
2,"[[0.2886751345948129, 0.2886751345948129, 0.2886751345948129, 0.2886751345948129, 0.288675134594...","[[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]]",2023,"[[0.5, 0.5, 0.5, 0.5]]",1,"[[0.038604571824109146, 0.07720914364821829, 0.07720914364821829, 0.038604571824109146, 0.038604...","[[0.09245003270420485, 0.3698001308168194, 0.1849000654084097, 0.09245003270420485, 0.0924500327..."


In [92]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12547 entries, 0 to 12546
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   authors        12547 non-null  object
 1   title          12547 non-null  object
 2   year           12547 non-null  int64 
 3   sourcetitle    12547 non-null  object
 4   citedby        12547 non-null  int64 
 5   abstract       12547 non-null  object
 6   indexkeywords  12547 non-null  object
dtypes: int64(2), object(5)
memory usage: 686.3+ KB


In [95]:
from sklearn.cluster import KMeans
ss = []
for i in range (1,21):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(articles_df['indexkeywords'])
    ss.append(kmeans.inertia_)
plt.plot(range(5,21),ss)
plt.show()

ValueError: setting an array element with a sequence.